In [1]:
import json
import os
import openai
import pandas as pd
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
def get_prompt_financial():
    return '''Please retrieve company name, revenue, net income and earnings per share (a.k.a. EPS)
    from the following news article. If you can't find the information from this article 
    then return "". Do not make things up.    
    Then retrieve a stock symbol corresponding to that company. For this you can use
    your general knowledge (it doesn't have to be from this article). Always return your
    response as a valid JSON string. The format of that string should be this, 
    {
        "Company Name": "Walmart",
        "Stock Symbol": "WMT",
        "Revenue": "12.34 million",
        "Net Income": "34.78 million",
        "EPS": "2.1 $"
    }
    News Article:
    ============

    '''

In [4]:
text = '''
    Tesla's Earning news in text format: Tesla's earning this quarter blew all the estimates. They reported 4.5 billion $ profit against a revenue of 30 billion $. Their earnings per share was 2.3 $
    '''
    

prompt = get_prompt_financial() + text
response = openai.ChatCompletion.create( # type: ignore
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user","content": prompt
        }
    ]
)


In [5]:
print(response)

{
  "id": "chatcmpl-7ZOl2mYYmT7k09ZrUWM8C4FS9N1SU",
  "object": "chat.completion",
  "created": 1688671004,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n    \"Company Name\": \"Tesla\",\n    \"Stock Symbol\": \"TSLA\",\n    \"Revenue\": \"30 billion $\",\n    \"Net Income\": \"4.5 billion $\",\n    \"EPS\": \"2.3 $\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 220,
    "completion_tokens": 51,
    "total_tokens": 271
  }
}


In [6]:
content = response.choices[0]['message']['content'] # type: ignore
print(content)


{
    "Company Name": "Tesla",
    "Stock Symbol": "TSLA",
    "Revenue": "30 billion $",
    "Net Income": "4.5 billion $",
    "EPS": "2.3 $"
}


In [7]:

try:
    data = json.loads(content)
    df = pd.DataFrame(data.items(), columns=['Measure', 'Value'])
    print(df)
except (json.decoder.JSONDecodeError, IndexError):
    pd.DataFrame({
        "Measure": ['Company Name', "Stock Symbol", "Revenue", "Net Income", "EPS"],
        "Value": ["", "", "", "", ""]
    })

        Measure          Value
0  Company Name          Tesla
1  Stock Symbol           TSLA
2       Revenue   30 billion $
3    Net Income  4.5 billion $
4           EPS          2.3 $
